
## Instalar las dependencias y configurar el entorno

In [ ]:
!pip install pandas-datareader

## Importar las dependencias del proyecto

In [ ]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [ ]:
tf.__version__

'2.3.0'

## Construir la red neuronal de la AI del Trader 

In [ ]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Manten, Compra, Vende
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    
    self.model = self.model_builder()
    
  def model_builder(self):
    
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))
    
    return model
  
  def trade(self, state):
    
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  
  def batch_train(self, batch_size):
    
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      reward = reward
      q_optimal=reward
      if not done:
        #q*(s,a)=r+gamma*max(q(s',a'))
        q_optimal = reward + self.gamma * np.amax(self.model.predict(next_state)[0])   
        
      target = self.model.predict(state)  # 0.6  0.8  0.1    
      target[0][action] = q_optimal          # 0.9  0.8  0.1
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

In [ ]:
tensor=tf.constant(np.arange(3).reshape(1,3))
print(tensor[0])


tf.Tensor([0 1 2], shape=(3,), dtype=int64)


In [ ]:
np.random.choice(2,10,p = [ 0.2 ,0.8 ])

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1])

## Paso 4: Pre procesado del dataset

### Definir las funciones adicionales

#### Sigmoide

In [ ]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

#### Función de formato de precios

In [ ]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

#### Carga del dataset

In [ ]:
def dataset_loader(stock_name):
  
  
  dataset = data_reader.DataReader(stock_name, data_source="yahoo")
  
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  
  close = dataset['Close']

  return close

### State creator

In [ ]:
def state_creator(data, timestep, window_size):
  
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

### Cargar una divisa de mercado

In [ ]:
stock_name = "AAPL"
data = dataset_loader(stock_name)

In [ ]:
data.head()

Date
2015-09-15    29.070000
2015-09-16    29.102501
2015-09-17    28.480000
2015-09-18    28.362499
2015-09-21    28.802500
Name: Close, dtype: float64

## Paso 5: Entrenar la AI Trader

### Configurar los hyper parámetros

In [ ]:
window_size = 10
episodes = 1000

batch_size = 32
data_samples = len(data) - 1

### Definir el modelo del AI Trader

In [ ]:
trader = AI_Trader(window_size)

In [ ]:
trader.model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                352       
_________________________________________________________________
dense_5 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_6 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


### Bucle de entrenamiento

In [ ]:
for episode in range(1, episodes + 1):
  
  print("Episodio: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    
    if action == 1: #Compra
      trader.inventory.append(data[t])
      print("AI Trader compró: ", stocks_price_format(data[t]))
      
    elif action == 2 and len(trader.inventory) > 0: #Vende
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader vendió: ", stocks_price_format(data[t]), " Beneficio: " + stocks_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("BENEFICIO TOTAL: {}".format(total_profit))
      print("########################")
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))
    

  0%|          | 0/1257 [00:00<?, ?it/s]

Episodio: 1/1000
AI Trader compró:  $ 29.102501
AI Trader compró:  $ 28.480000
AI Trader vendió:  $ 28.802500  Beneficio: - $ 0.300001
AI Trader compró:  $ 28.350000
AI Trader compró:  $ 28.750000
AI Trader vendió:  $ 28.110001  Beneficio: - $ 0.369999
AI Trader vendió:  $ 27.264999  Beneficio: - $ 1.085001
AI Trader vendió:  $ 27.575001  Beneficio: - $ 1.174999
AI Trader compró:  $ 28.030001
AI Trader compró:  $ 27.900000
AI Trader compró:  $ 27.965000
AI Trader compró:  $ 27.760000
AI Trader vendió:  $ 27.932501  Beneficio: - $ 0.097500
AI Trader compró:  $ 28.442499
AI Trader vendió:  $ 28.875000  Beneficio: $ 0.975000
AI Trader compró:  $ 28.820000
AI Trader vendió:  $ 29.817499  Beneficio: $ 1.852499
AI Trader vendió:  $ 30.132500  Beneficio: $ 2.372499


  3%|▎         | 34/1257 [00:07<19:32,  1.04it/s]

AI Trader compró:  $ 30.295000


  3%|▎         | 35/1257 [00:10<31:03,  1.52s/it]

...

 95%|█████████▌| 1195/1257 [56:40<02:57,  2.86s/it]

AI Trader vendió:  $ 85.747498  Beneficio: $ 17.434998


 95%|█████████▌| 1196/1257 [56:43<02:56,  2.90s/it]

AI Trader vendió:  $ 88.019997  Beneficio: $ 16.257500


KeyboardInterrupt: ignored